In [1]:
import os
import cv2
import dlib
import numpy as np

In [2]:
import os
import cv2
import dlib
import numpy as np

# Paths for input and output
fake_videos_dir = "F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end"

output_fake_faces = "E:/dataset_1/happy/fake"
output_optical_flow = "E:/dataset_1/happy/optical_flow"
output_edges = "E:/dataset_1/happy/edges"

# Create directories if not exist
for path in [output_fake_faces, output_optical_flow, output_edges]:
    os.makedirs(path, exist_ok=True)

# Load face detector
detector = dlib.get_frontal_face_detector()
predictor_path = "D:/school/Research/TC-FET-Deepfake-Detector/preprocessing/shape_predictor_68_face_landmarks.dat"  
predictor = dlib.shape_predictor(predictor_path)

def compute_dense_optical_flow(prev_frame, next_frame):
    """Computes dense optical flow and converts it into an RGB representation."""
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    hsv = np.zeros_like(prev_frame)
    hsv[..., 1] = 255
    hsv[..., 0] = angle * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def extract_edges(frame):
    """Extracts edges from a frame using the Canny edge detection method."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def process_video(video_path, save_faces_dir, save_flow_dir, save_edges_dir):
    """Extracts faces and ensures optical flow and edge maps match the face frames."""
    folder_name = os.path.basename(os.path.dirname(video_path))  # Get folder name for naming
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    frame_count = 0

    # Counters for statistics
    face_count = 0
    flow_count = 0
    edge_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        if faces:  # Only process optical flow and edges if a face is found
            for i, face in enumerate(faces):
                landmarks = predictor(gray, face)
                interocular_distance = landmarks.part(42).x - landmarks.part(39).x

                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                size = 5 * interocular_distance  # Increase bounding box size

                x1 = max(0, x - size)
                y1 = max(0, y - size)
                x2 = min(frame.shape[1], x + size)
                y2 = min(frame.shape[0], y + size)

                face_crop = frame[y1:y2, x1:x2]

                if face_crop.shape[0] > 0 and face_crop.shape[1] > 0:  # Ensure valid cropping
                    face_resized = cv2.resize(face_crop, (299, 299))

                    filename_base = f"{folder_name}_{os.path.basename(video_path)}_frame{frame_count}_face{i}"
                    filename_face = f"{filename_base}.jpg"
                    filename_flow = f"{filename_base}_flow.jpg"
                    filename_edges = f"{filename_base}_edges.jpg"

                    # Save face
                    cv2.imwrite(os.path.join(save_faces_dir, filename_face), face_resized)
                    face_count += 1

                    # Compute optical flow and crop to face region
                    if prev_frame is not None:
                        flow_map = compute_dense_optical_flow(prev_frame, frame)
                        flow_crop = flow_map[y1:y2, x1:x2]
                        if flow_crop.shape[0] > 0 and flow_crop.shape[1] > 0:
                            flow_resized = cv2.resize(flow_crop, (299, 299))
                            cv2.imwrite(os.path.join(save_flow_dir, filename_flow), flow_resized)
                            flow_count += 1

                    # Compute edges and crop to face region
                    edge_map = extract_edges(frame)
                    edge_crop = edge_map[y1:y2, x1:x2]
                    if edge_crop.shape[0] > 0 and edge_crop.shape[1] > 0:
                        edge_resized = cv2.resize(edge_crop, (299, 299))
                        cv2.imwrite(os.path.join(save_edges_dir, filename_edges), edge_resized)
                        edge_count += 1

        prev_frame = frame
        frame_count += 1

    cap.release()

    # Print statistics for this video
    print(f"\n Video Processed: {os.path.basename(video_path)}")
    print(f"    Faces Extracted: {face_count}")
    print(f"    Optical Flow Maps: {flow_count}")
    print(f"    Edge Maps: {edge_count}")
    print("=" * 50)

def traverse_and_process(root_dir, save_faces_dir, save_flow_dir, save_edges_dir, check_happy=False):
    """Traverses folders recursively, filtering based on 'happy' if required, and skips already processed videos."""
    for root, _, files in os.walk(root_dir):
        if check_happy and "happy" not in root.lower():
            continue  # Skip this folder if 'happy' is not in its path
        # if check_happy and all(w not in root.lower() for w in ["w0", "w1"]):
        #     continue  # Skip this folder if 'w0' || 'w1' is not in its path
        
        for file in files:
            if file.endswith(".mp4") or file.endswith(".avi"):
                video_path = os.path.join(root, file)
                folder_name = os.path.basename(root)
                
                # Generate expected output filenames
                base_filename = f"{folder_name}_{file}_frame"
                
                # Check if at least one output exists
                processed = any(
                    filename.startswith(base_filename) 
                    for directory in [save_faces_dir, save_flow_dir, save_edges_dir] 
                    for filename in os.listdir(directory)
                )

                if processed:
                    print(f"Skipping {video_path} (already processed)")
                    continue

                print(f"Processing {video_path}...")
                process_video(video_path, save_faces_dir, save_flow_dir, save_edges_dir)

# Process fake videos (manipulated_videos) but only if 'happy' is in the path
traverse_and_process(fake_videos_dir, output_fake_faces, output_optical_flow, output_edges, check_happy=False)

print("\nProcessing complete!")


Processing F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end\711_M007.mp4...

 Video Processed: 711_M007.mp4
    Faces Extracted: 291
    Optical Flow Maps: 291
    Edge Maps: 291
Processing F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end\023_M113.mp4...

 Video Processed: 023_M113.mp4
    Faces Extracted: 552
    Optical Flow Maps: 552
    Edge Maps: 552
Processing F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end\759_W025.mp4...

 Video Processed: 759_W025.mp4
    Faces Extracted: 855
    Optical Flow Maps: 855
    Edge Maps: 855
Processing F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end\952_W101.mp4...

 Video Processed: 952_W101.mp4
    Faces Extracted: 699
    Optical Flow Maps: 699
    Edge Maps: 699
Processing F:/dataset/DeeperForensics-1.0/manipulated_videos/end_to_end\351_M023.mp4...

 Video Processed: 351_M023.mp4
    Faces Extracted: 683
    Optical Flow Maps: 683
    Edge Maps: 683
Processing F:/dataset/DeeperForensics-1.0/man